In [0]:
-- =========================================================================================
-- TRT Raw Clean Table with Treatment Type Information
-- =========================================================================================
-- Integrates treatment (TRT) data with treatment type descriptions from bltrt
-- =========================================================================================

CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

-- =========================================================================================
-- Main Table Creation with Treatment Type CTE
-- =========================================================================================

--CREATE OR REPLACE TABLE teamconnect.pca_prism.06_tbl_trt_raw_clean AS
INSERT OVERWRITE TABLE teamconnect.pca_prism.06_tbl_trt_raw_clean
WITH cte_bltrt AS (
  SELECT
    CONCAT(trtmex, trtmln) AS btn,
    env AS dpi_environment,
    trtdat AS treatment_date,
    trttyp AS treatment_type,
    CASE trttyp
      WHEN 'BPD' THEN 'Bill Past Due (Treatable in WV only)'
      WHEN 'COL' THEN 'Cut Off List'
      WHEN 'CVD' THEN 'COVID-19 Override'
      WHEN 'DFD' THEN 'NY Data Feed'
      WHEN 'DPA' THEN 'Deferred Payment Arrangement'
      WHEN 'DSC' THEN 'Suspend Non-Payment (2-way)'
      WHEN 'DSH' THEN 'Dish Service - Dish Only Suspend'
      WHEN 'EMR' THEN 'Collections Supervisor Use Only for Emergency (i.e. Natural Disasters)'
      WHEN 'MED' THEN 'Medical Override'
      WHEN 'NBS' THEN 'Non-Basic Suspend'
      WHEN 'NRS' THEN 'Non-Regulated Suspend'
      WHEN 'NSF' THEN 'Non-Sufficient Fund Check'
      WHEN 'NVS' THEN 'Video service (FiberOptic or Frontier TV)'
      WHEN 'OWD' THEN 'One-Way Suspend (NY Only)'
      WHEN 'OVR' THEN 'Treatment Override'
      WHEN 'PWO' THEN 'Pre-Write Off'
      WHEN 'PYX' THEN 'Payment Extension'
      WHEN 'REM' THEN 'Reminder Notice'
      WHEN 'RTN' THEN 'Return Notice'
      WHEN 'TBK' THEN 'Toll Suspend'
      WHEN 'TTL' THEN 'Treatment Termination Letter'
      ELSE 'UNKNOWN'
    END AS treatment_type_description,
    trtamt AS treatment_amount
  FROM it_bronze_prod.dpi.bltrt
)

SELECT
  -- Core BTN/Account Info
  CASE 
    WHEN LENGTH(exc_digits) = 6 AND LENGTH(lne_digits) BETWEEN 1 AND 4
      THEN CONCAT(exc_digits, LPAD(lne_digits, 4, '0'))
    ELSE NULL
  END AS btn,
  t.dpi_environment,
  
  -- TRT Fields (Treatment Data from csacttrt)
  remdte, 
  remamt,
  spsdte, 
  spsamt, 
  opnsps,
  nsfdte, 
  opnnsf,
  dpadte, 
  opndpa,
  ovrdte, 
  opnovr,
  pyxdte,
  sedqupsl,
  
  -- Treatment Type Fields (from bltrt CTE)
  trt.treatment_date,
  trt.treatment_type,
  trt.treatment_type_description,
  trt.treatment_amount

FROM (
  SELECT
    REGEXP_REPLACE(CAST(actexc AS STRING),'[^0-9]','') AS exc_digits,
    REGEXP_REPLACE(CAST(actlne AS STRING),'[^0-9]','') AS lne_digits,
    UPPER(TRIM(env)) AS dpi_environment,
    
    clean_date(COALESCE(
      TRY_TO_DATE(CAST(remdte AS STRING),'yyyy-MM-dd'),
      TRY_TO_DATE(CAST(remdte AS STRING),'yyyyMMdd'),
      TRY_TO_DATE(CAST(remdte AS STRING),'MM/dd/yyyy')
    )) AS remdte,
    
    clean_date(COALESCE(
      TRY_TO_DATE(CAST(spsdte AS STRING),'yyyy-MM-dd'),
      TRY_TO_DATE(CAST(spsdte AS STRING),'yyyyMMdd'),
      TRY_TO_DATE(CAST(spsdte AS STRING),'MM/dd/yyyy')
    )) AS spsdte,
    
    clean_date(COALESCE(
      TRY_TO_DATE(CAST(nsfdte AS STRING),'yyyy-MM-dd'),
      TRY_TO_DATE(CAST(nsfdte AS STRING),'yyyyMMdd'),
      TRY_TO_DATE(CAST(nsfdte AS STRING),'MM/dd/yyyy')
    )) AS nsfdte,
    
    clean_date(COALESCE(
      TRY_TO_DATE(CAST(dpadte AS STRING),'yyyy-MM-dd'),
      TRY_TO_DATE(CAST(dpadte AS STRING),'yyyyMMdd'),
      TRY_TO_DATE(CAST(dpadte AS STRING),'MM/dd/yyyy')
    )) AS dpadte,
    
    clean_date(COALESCE(
      TRY_TO_DATE(CAST(ovrdte AS STRING),'yyyy-MM-dd'),
      TRY_TO_DATE(CAST(ovrdte AS STRING),'yyyyMMdd'),
      TRY_TO_DATE(CAST(ovrdte AS STRING),'MM/dd/yyyy')
    )) AS ovrdte,
    
    clean_date(COALESCE(
      TRY_TO_DATE(CAST(pyxdte AS STRING),'yyyy-MM-dd'),
      TRY_TO_DATE(CAST(pyxdte AS STRING),'yyyyMMdd'),
      TRY_TO_DATE(CAST(pyxdte AS STRING),'MM/dd/yyyy')
    )) AS pyxdte,
    
    CAST(remamt AS DECIMAL(18,2)) AS remamt,
    CAST(spsamt AS DECIMAL(18,2)) AS spsamt,
    
    CASE WHEN UPPER(TRIM(CAST(opnsps AS STRING))) IN ('Y','1') THEN 1
         WHEN UPPER(TRIM(CAST(opnsps AS STRING))) IN ('N','0') THEN 0
         ELSE TRY_CAST(opnsps AS INT) END AS opnsps,
    
    CASE WHEN UPPER(TRIM(CAST(opnnsf AS STRING))) IN ('Y','1') THEN 1
         WHEN UPPER(TRIM(CAST(opnnsf AS STRING))) IN ('N','0') THEN 0
         ELSE TRY_CAST(opnnsf AS INT) END AS opnnsf,
    
    CASE WHEN UPPER(TRIM(CAST(opndpa AS STRING))) IN ('Y','1') THEN 1
         WHEN UPPER(TRIM(CAST(opndpa AS STRING))) IN ('N','0') THEN 0
         ELSE TRY_CAST(opndpa AS INT) END AS opndpa,
    
    CASE WHEN UPPER(TRIM(CAST(opnovr AS STRING))) IN ('Y','1') THEN 1
         WHEN UPPER(TRIM(CAST(opnovr AS STRING))) IN ('N','0') THEN 0
         ELSE TRY_CAST(opnovr AS INT) END AS opnovr,
    
    SEDQUPSL AS sedqupsl
    
  FROM it_bronze_prod.dpi.csacttrt
) t

-- LEFT JOIN to treatment type data (keeps all TRT records even if no treatment type found)
LEFT JOIN cte_bltrt trt
  ON CONCAT(t.exc_digits, LPAD(t.lne_digits, 4, '0')) = trt.btn
  AND t.dpi_environment = trt.dpi_environment

WHERE LENGTH(exc_digits) = 6
  AND LENGTH(lne_digits) BETWEEN 1 AND 4
  AND CONCAT(exc_digits, LPAD(lne_digits,4,'0')) RLIKE '^[0-9]{10}$';

-- =========================================================================================
-- Optimize the table
-- =========================================================================================

OPTIMIZE teamconnect.pca_prism.06_tbl_trt_raw_clean
ZORDER BY (btn, dpi_environment);